In [44]:
import subprocess
from pathlib import Path
import tempfile
import shutil
from tqdm import tqdm

In [45]:
dir = '/home/nikola/projects/data/inputs/spanish-salish-admirality_trail'

In [47]:
def organize_files_by_extension(directory_path):
    directory = Path(directory_path)

    # Ensuring the directory exists
    if not directory.is_dir():
        raise ValueError(f"The path {directory_path} is not a valid directory.")

    # Getting a list of files
    files = [file for file in directory.iterdir() if file.is_file()]

    # Using tqdm for progress bar
    for file in tqdm(files, desc="Organizing files"):
        # Get file extension in lower case and create a subdirectory name
        extension = file.suffix.lower()[1:]  # remove the leading dot and convert to lower case
        if extension:  # skip files without an extension
            subdirectory = directory / extension

            # Create the subdirectory if it does not exist
            subdirectory.mkdir(exist_ok=True)

            # Move the file
            shutil.move(str(file), str(subdirectory / file.name))

In [31]:
organize_files_by_extension(dir)

Organizing files:   0%|          | 0/3448 [00:00<?, ?it/s]

Organizing files: 100%|██████████| 3448/3448 [00:00<00:00, 29269.12it/s]


In [32]:
def rename_files_sequentially(directory_path):
    directory = Path(directory_path)

    # Ensuring the directory exists
    if not directory.is_dir():
        raise ValueError(f"The path {directory_path} is not a valid directory.")

    # Getting a list of files and sorting them by name
    files = sorted([file for file in directory.iterdir() if file.is_file()], key=lambda x: x.name)

    # Using tqdm for progress bar
    for i, file in enumerate(tqdm(files, desc="Renaming files")):
        # Construct new file name with same extension
        new_name = f"{i:04d}{file.suffix}"

        # Renaming the file
        file.rename(directory / new_name)

In [33]:
rename_files_sequentially(dir + '/jpg')
rename_files_sequentially(dir + '/dng')

Renaming files: 100%|██████████| 1721/1721 [00:00<00:00, 48977.80it/s]


Renaming files: 100%|██████████| 1721/1721 [00:00<00:00, 45530.73it/s]


In [56]:
def create_video_from_images(directory_path, output_filename, frame_rate=24):
    directory = Path(directory_path)

    # Check if directory exists
    if not directory.is_dir():
        raise ValueError(f"The path {directory_path} is not a valid directory.")

    # Supported image formats (in lower case)
    image_formats = ['jpg', 'jpeg', 'png']

    # Temporary directory to store renamed images
    with tempfile.TemporaryDirectory() as temp_dir:
        temp_dir_path = Path(temp_dir)

        # Finding and copying images
        image_files = []
        for file in directory.iterdir():
            if file.is_file() and file.suffix.lower()[1:] in image_formats:
                image_files.append(file)

        # Sort and rename images
        for i, file in enumerate(sorted(image_files, key=lambda x: x.name)):
            temp_file = temp_dir_path / f"{i:04d}{file.suffix.lower()}"  # Using lower case for consistency
            shutil.copy(file, temp_file)

        # Construct FFmpeg command
        command = [
            'ffmpeg',
            '-framerate', str(frame_rate),
            '-i', str(temp_dir_path / '%04d.jpg'),  # Using .jpg in pattern, as FFmpeg treats the suffix in the pattern as a placeholder
            '-c:v', 'libx264',
            '-pix_fmt', 'yuv420p',
            str(directory / output_filename)
        ]

        # Run the FFmpeg command
        subprocess.run(command)

In [57]:
create_video_from_images(dir + '/jpg', "output.mp4", frame_rate=12)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e